In [1]:
import osmnx as ox
import networkx as nx
import pandas as pd
import json
import requests
import random

from rtree import index
from shapely.geometry import box
from utils import *
from collections import defaultdict

In [2]:
G = ox.save_load.load_graphml(filename="Singapore_drive.graphml")
G_ped = ox.save_load.load_graphml(filename="Singapore_walk.graphml")
nodes_central = pd.read_pickle('data/Central_nodes.pkl')
nodes_central.drop([7330643916], inplace=True)
nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)

print(type(G))

<class 'networkx.classes.multidigraph.MultiDiGraph'>


In [3]:
# Calculate congestion of network for a time slice

f = open("../../Traffic speed bands/Fri Feb 14 09_06_16 2020.json", "r").read()
x = json.loads(f)
speed_bands = pd.DataFrame.from_dict(x, orient='columns')
process_speed_band(speed_bands)
calculate_congestion(G, edges, speed_bands)

edges.head()

/home/orionstar25/Semester-8-Major-Project/Open Street Maps/Phase 5 - System Optimal Social Model/utils.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MaximumSpeed'][i] = '80'


,u,v,key,osmid,oneway,name,highway,maxspeed,length,travel_time,Location,geometry,observed_speed,capacity
0,1820000257,1865254946,0,174765824,True,Tampines Avenue 8,tertiary,60.0,14.248,0.85488,"(1.3565805, 103.9326163, 1.3566943, 103.9326752)","LINESTRING (103.93262 1.35658, 103.93268 1.35669)",31.5,4
1,1820000257,4662056865,0,587044201,True,Tampines Avenue 5,primary,60.0,27.282,1.63692,"(1.3565805, 103.9323996, 1.3566955, 103.9326163)","LINESTRING (103.93262 1.35658, 103.93240 1.35670)",31.5,8
2,3874553858,6995781882,0,384254693,False,Fidelio Street,tertiary,40.0,20.740,1.8666,"(1.3191823, 103.9212368, 1.3192215, 103.9214192)","LINESTRING (103.92124 1.31918, 103.92142 1.31922)",34,9
3,3874553858,3874553861,0,384254631,True,Fidelio Street,tertiary,40.0,15.191,1.3671900000000001,"(1.3191111, 103.9211207, 1.3191823, 103.9212368)","LINESTRING (103.92124 1.31918, 103.92112 1.31911)",34,7
4,1820000259,245190289,0,170861145,True,Tampines Avenue 8,tertiary,60.0,13.573,0.8143800000000001,"(1.3565404, 103.9326858, 1.3566471, 103.9327451)","LINESTRING (103.93275 1.35665, 103.93269 1.35654)",31.5,4


In [4]:
# Fill in all nodes to find neighbours
# key: node label ,value: location
idx = index.Index()

xx = nodes['x'].astype(float)
yy = nodes['y'].astype(float)
node_id = nodes['osmid']

for node, x, y in zip(node_id, xx, yy):
    idx.insert(int(node), (x,y,x,y))

In [5]:
# NEED TO CHANGE INTO NEW SG.OSM FOR DIFFERENT TIME SLICE (BECAUSE OF BPR VALUES AS WEIGHT)
# ox.save_load.save_graph_shapefile(G, filename='graph')

In [6]:
def modified_hybrid_search(G, x):
    n = len(x)
    cutoff = math.floor(n/math.exp(1))
    min_pair = x[0]
    source = min_pair[0][0]
    dest = min_pair[0][1]
    min_tt = float('inf')

    for i in range(n):
        source_x = nodes[nodes['osmid'] == source]['x'].values[0]
        source_y = nodes[nodes['osmid'] == source]['y'].values[0]
        dest_x = nodes[nodes['osmid'] == dest]['x'].values[0]
        dest_y = nodes[nodes['osmid'] == dest]['y'].values[0]

        url = "http://0.0.0.0:5000/route/v1/driving/{},{};{},{}".format(source_x, source_y, dest_x, dest_y)
        r = requests.get(url)
        json = r.json()            
        travel_time = json['routes'][0]['weight']

        # have reached cutoff and didnt find optimal yet
        if i > cutoff:
            if travel_time < min_tt:
                min_tt = travel_time
                min_pair = x[i]
                break
            else:
                break
        else:
            if travel_time < min_tt:
                min_tt = travel_time
                min_pair = x[i]

                if i < n-1:
                    if travel_time < x[i+1][1]:
                        break
                    else:
                        source = x[i+1][0][0]
                        dest = x[i+1][0][1]
                else:
                    break
            else:
                if i < n-1:
                    if min_tt < x[i+1][1]:
                        break
                    else:
                        source = x[i+1][0][0]
                        dest = x[i+1][0][1]
                else:
                    break

    return min_tt, min_pair[0]

### All miles routing

In [7]:
# get first 10 sources from central SG
sources = nodes_central['osmid'].values[:25]
 
# get last 10 targets from central SG
targets = nodes_central['osmid'].values[-20:]

# Find optimal transit points for the middle mile
AD = []
demand = []
# all_miles = defaultdict(lambda: list())

for s in sources:
    for t in targets:
        print(s, t)
        AD.append([s,t])
        
        orig_point = nodes[nodes['osmid'] == s].geometry.values[0]
        target_point = nodes[nodes['osmid'] == t].geometry.values[0]
        orig_xy = (orig_point.y, orig_point.x)
        target_xy = (target_point.y, target_point.x)

        # Max distance for walking is 720m (10 min * 1.2 m/s)
        # Find all nodes within 720 m from candidate sources and candidate destinations
        candidate_dest = find_nearest_nodes(nodes, target_xy, 400, idx)
        candidate_source = find_nearest_nodes(nodes, orig_xy, 400, idx)

        if len(candidate_dest) == 0:
            candidate_dest.append(ox.get_nearest_node(G, target_xy))

        if len(candidate_source) == 0:
            candidate_source.append(ox.get_nearest_node(G, orig_xy))

        euclidean = get_length_dict(nodes, candidate_source, candidate_dest)
        _, OD = modified_hybrid_search(G, euclidean)
        demand.append(OD)
        
#         # first mile: X--->A Use only pedestrian network
#         fm_length, fm_path = nx.single_source_dijkstra(G_ped, s, OD[0], weight='length')
#         all_miles[(s,t)].append(fm_path)
        
#         # last mile: : B--->Y Use only pedestrian network
#         lm_length, lm_path = nx.single_source_dijkstra(G_ped, OD[1], t, weight='length')
#         all_miles[(s,t)].append(lm_path)

233783297 247037926
233783297 1931968487
233783297 247037929
233783297 4604559340
233783297 5015699438
233783297 4594335727
233783297 247037935
233783297 4594335729
233783297 247037936
233783297 247037937
233783297 571883509
233783297 4604559345
233783297 1931968496
233783297 4604559352
233783297 1931968501
233783297 1398079475
233783297 4594335734
233783297 1931968508
233783297 376619005
233783297 1363263487
626491396 247037926
626491396 1931968487
626491396 247037929
626491396 4604559340
626491396 5015699438
626491396 4594335727
626491396 247037935
626491396 4594335729
626491396 247037936
626491396 247037937
626491396 571883509
626491396 4604559345
626491396 1931968496
626491396 4604559352
626491396 1931968501
626491396 1398079475
626491396 4594335734
626491396 1931968508
626491396 376619005
626491396 1363263487
5015699464 247037926
5015699464 1931968487
5015699464 247037929
5015699464 4604559340
5015699464 5015699438
5015699464 4594335727
5015699464 247037935
5015699464 4594335729
5

1363263536 247037935
1363263536 4594335729
1363263536 247037936
1363263536 247037937
1363263536 571883509
1363263536 4604559345
1363263536 1931968496
1363263536 4604559352
1363263536 1931968501
1363263536 1398079475
1363263536 4594335734
1363263536 1931968508
1363263536 376619005
1363263536 1363263487
1363263538 247037926
1363263538 1931968487
1363263538 247037929
1363263538 4604559340
1363263538 5015699438
1363263538 4594335727
1363263538 247037935
1363263538 4594335729
1363263538 247037936
1363263538 247037937
1363263538 571883509
1363263538 4604559345
1363263538 1931968496
1363263538 4604559352
1363263538 1931968501
1363263538 1398079475
1363263538 4594335734
1363263538 1931968508
1363263538 376619005
1363263538 1363263487
6938787891 247037926
6938787891 1931968487
6938787891 247037929
6938787891 4604559340
6938787891 5015699438
6938787891 4594335727
6938787891 247037935
6938787891 4594335729
6938787891 247037936
6938787891 247037937
6938787891 571883509
6938787891 4604559345
693878

In [8]:
# Convert to .tntp files for Frank Wolfe
# make_nodes_tntp(nodes)
# make_net_tntp(edges)
make_trips_tntp(demand)

No. of requests:  2884


In [1]:
# Start system-centric Frank Wolfe implementation <make this alright later>
import Frank_Wolfe as FW

directory = "data/Singapore_"
link_file = '{}net.tntp'.format(directory)
trip_file = '{}trips_70.tntp'.format(directory)
node_file = '{}node.tntp'.format(directory)
SO = True

fw = FW.Run(link_file, trip_file, node_file, SO)
fw.showODPath()

In [ ]:
# Integrate middle mile into miles dictionary
# TODO <maybe later cuz not needed rn>